<a href="https://colab.research.google.com/github/jasperSha/cloud_music/blob/main/model/neuralnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
!gcloud config set project cluster-music

In [2]:
import os
# # create virtual directory for image data
# os.makedirs('song_images', exist_ok=True)

# create virtual directory for frequency data
os.makedirs('song_freqs', exist_ok=True)

In [ ]:
'''
notes:
add user pref to list(dims) orthogonal to every other dimensional
pull not push

euclidean distance
batchsize=10
i like 4 songs
find centroid of the 4 songs (centroid of n-features)
reduce distance to centroid of 4 songs by some measure (lr=alpha)

i dislike 6 songs
find centroid of 4 songs
increase distance to centroid
same shit

upload song_id's to main model after going through the (batchsize) songs
update song_id distances in main model

upload new n-vector distances, then rerun kmeans for each 100th new user or something

n samples of each k cluster initial state
users like songs, find centroid of liked songs, shift user towards centroid
after number of batches of shifting user, begin shifting song locations by preferences
then do feedback loop of both batch and user shifting locations

build function for querying initial songs, songs from each cluster, and updating location of each using gradient descent

NOTE:
each user has list of root nodes that refer to their different access points to various clusters
if a user likes 






'''
# Notes and Potential Issues:
# - to retrieve song data from id alone requires a search in fullData which may be computationally expensive
# - same point as above but this time pertaining to updating values in fullData
# - additionally, simply sending the fullData as a parameter to the function may be expensive
# - subfunctions could easily be made but were avoided due to the passing of large parameters

# Updates the server model with size n songs from a user client
# Parameters:
# fullData - Euclidean space representation of all songs on the server
# userUpdate - n x 2 array of [songId][preference] where preference = 1 is liked and 0 is disliked
# alpha - learning rate for moving songs in the server data Euclidean space


# TODO:
#  - allow for user to provide centroid
#  - create user client functrion for updating location
#  - create placement batch
#  - set blocker for user client until acclimation period is over


def serverModelUpdate(fullData, userUpdate, alpha):
  # splits user songs into liked and disliked lists
  pull = []
  push = []
  for song in userUpdate:
    if(song[1]):
      pull.append(song[0])
    else:
      push.append(song[0])
  
  # creates and finds the centroid for liked songs
  centroid = [0] * (fullData.shape[1] - 1)
  for id in pull:
    songData = fullData.loc[fullData['id'] == id]
    for i in range(len(songData)):
      centroid[i] += songData[i]
  centroid = [value / len(push) for value in centroid]

  # applies the liked song update to the server data
  for id in pull:
    serverSongData = fullData.loc[fullData['id']==id]
    for i in range(len(serverSongData)):
      if(i==0):         # poor fix to skip the id data
        continue
      serverSongData[i] = (centroid[i] - serverSongData[i]) * alpha + serverSongData[i]
    fullData.loc[fullData['id']==id] = serverSongData
  
  # same code for centroid of disliked songs
  centroid = [0] * (fullData.shape[1] - 1)
  for id in push:
    songData = fullData.loc[fullData['id'] == id]
    for i in range(len(songData)):
      centroid[i] += songData[i]
  centroid = [value / len(push) for value in centroid]

  # same code for updating server data with disliked songs
  for id in push:
    serverSongData = fullData.loc[fullData['id']==id]
    for i in range(len(serverSongData)):
      if(i==0):         # poor fix to skip the id data
        continue
      serverSongData[i] = (centroid[i] - serverSongData[i]) * alpha + serverSongData[i]
    fullData.loc[fullData['id']==id] = serverSongData

class User:
  '''
  A user who likes multiple genres
  '''
  def __init__(self, user_id, base_model):
    self.user_id = user_id
  
  def set_batch(self, batch):
    '''
    @param batch: 
    json of {
      song_id: boolean
    }
    '''
    self.current_batch = batch
  
  def vote_songs(self, votes):
    '''
    @param votes: list of upvotes and downvotes e.g. [0, 1, 0, 1, 1]
    vote on all songs in current batch
    '''
    idx = 0
    for k, v in self.current_batch.items():
      self.current_batch[k] = votes[idx]
      idx += 1

  def get_batch(self):
    '''
    get batch votes
    '''
    return self.current_batch

class Model:
  '''
  Base recommendation model for a new user

  attributes:
    roots: list of root nodes, one from each large scale k cluster of all samples

  class Node:
    def __init__(self, root=True):
      if root:
        self.likes = 0
        self.dislikes = 0
      else:
        self.likes = INT_MAX
        self.dislikes = 0

    def get_ratio(self):
      return self.likes / self.dislikes

  # initial sweep
  for root in current_roots:
    run knn on root # query server model/data
    ask user for like/dislike/none on each neighbor
    if user likes song:
      song_leaf_node = Node(root=False) # here we set its ratio to inf
      add song_leaf_node to root.children
      root.likes += 1
    if user dislikes song:
      # don't add
      root.dislikes += 1
    if root.likes == 0 and root.dislikes != 0: # didn't like any song, and dislikes
      current_roots.remove(root) # drop this root and continue to other roots
      continue

  # update ratio priorities
  for root in current_roots:
    ratios = []
    get ratio of each root, append to ratios
    sort(ratios) # now we have priority queue of high ratio(lots of likes) songs
  
  # sweep through leaves by priority
  # same as initial sweep but this time roots are ordered in terms of their ratios
  for node in ratios:
    if node.children is None: # it's a leaf
      run knn on node, get neighbors
      ask for user input
    if node.children is not None:
      run_sweep(node.children)

  

  if root.e:
    continue to next root

  for node in root.children:
    run knn on node
    ask user for ... on each neighbor
    if like --> connect 


  after several arounds:
    for each tree (growing out from each root node)
    find node with most edges, set that as new root node(in list of root nodes)

  '''



class Server:
  '''
  list of users(ids: location, previous_songs)
  list of songs(locations)

  list of lists(k clusters)

  learning_rate

  gradient descent for user
  gradient descent for songs sent by user(liked/disliked)

  -1. let user choose initial genre/artist/songs
  0. start user in center of all songs
  1. pass user n of k nearest clusters (k << N, where N is total number of clusters in universe)
    [1, 1, 1, 2, 2, 2, 3, 3, 3]
  2. user likes/dislikes songs

  3. user sends likes/dislikes to server
  4. server gradient descents user
  5. Recheck the k nearest clusters
  6. sample n of those k nearest clusters
  ~ repeat 3 to 6

  0. 


  '''



In [ ]:
# load metadata
!gsutil cp gs://deepclustermusic/metadata/fullmeta.csv .

# # load images
# !gsutil cp gs://deepclustermusic/song_images './song_images'

# load frequency data
!gsutil -m cp -n gs://deepclustermusic/song_freqs/*.csv './song_freqs' &> /dev/null

In [33]:
import glob

import pandas as pd
import numpy as np

from sklearn.cluster import KMeans
from sklearn import decomposition
import plotly.graph_objects as go

metadata = pd.read_csv('fullmeta.csv')
metadata.drop(columns=['duration'], inplace=True) # already have duration in frequency measures

freqs = list(glob.glob('./song_freqs/*.csv'))
df = pd.concat((pd.read_csv(f) for f in freqs))

In [5]:
df = df.merge(metadata, how='left', on='id')

In [ ]:
df.head()

In [17]:
full_df = df.copy()
full_df.drop(columns=['sr', 'channel'], inplace=True, axis=1)

In [ ]:
full_df.head()

In [18]:
# normalize everything besides metadata
norm_columns = full_df.drop(columns=['id', 'artist', 'title']).columns
full_df[norm_columns] = (full_df[norm_columns] - full_df[norm_columns].min())/(full_df[norm_columns].max() - full_df[norm_columns].min())

In [38]:
full_df.shape

(16521, 32)

In [19]:
# PCA --> ndim = 3
pca = decomposition.PCA(n_components=4)
pca.fit(full_df[norm_columns])

PCA(copy=True, iterated_power='auto', n_components=4, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)

In [21]:
pca_df = full_df.copy()
pca_columns = ['x', 'y', 'z', 'w']
pca_df[pca_columns] = pca.transform(full_df[norm_columns])

In [22]:
pca_df.drop(columns=norm_columns, inplace=True)
pca_df.head()

,id,artist,title,x,y,z,w
0,WiQTC0ETRPU,Auburn,Auburn - Mii Hood (Remiix),0.170983,-0.072396,-0.042233,0.184619
1,WOHPuY88Ry4,Johnny Cash,Johnny Cash - A Boy Named Sue (Live at San Que...,-0.581605,0.010752,0.070999,0.048827
2,WAcJBBFBLKQ,Blueridge,My Lord's Going To Set Me Free,-0.085820,-0.296776,0.033634,-0.155827
3,Vw0vhiRtLkw,ShuuKaRen,ShuuKaRen 『UNIVERSE』 【Beats by Dr.Dre「Show You...,0.198485,-0.155359,0.221302,0.199218
4,Wp71R4bO0qc,NaN,Karl Sanders - Dying Embers of the Aga Mass SS...,-0.300323,0.391125,0.154699,0.168448


In [23]:
kmeans = KMeans(n_clusters=150, random_state=0).fit(pca_df[pca_columns])

In [25]:
cluster_df = pca_df.copy()
cluster_df['cluster'] = kmeans.labels_

In [35]:
plot = go.Figure()
for c in list(cluster_df.cluster.unique()):
  plot.add_trace(go.Scatter3d(x = cluster_df[cluster_df.cluster == c]['x'],
                              y = cluster_df[cluster_df.cluster == c]['y'],
                              z = cluster_df[cluster_df.cluster == c]['z'],
                              mode = 'markers', marker_size = 2, marker_line_width=1,
                              text=['title'],
                              name = 'Cluster ' + str(c)))
plot.update_layout(width=800, height=800, autosize=True, showlegend=True,
                   scene=dict(xaxis=dict(title='x', titlefont_color='black'),
                              yaxis=dict(title='y', titlefont_color='black'),
                              zaxis=dict(title='z', titlefont_color='black')),
                   font=dict(family='Gilroy', color='black', size=12))

In [36]:
# note clusters 1 and 3 seem close to each other, whereas cluster 2 is further away
df1 = cluster_df[cluster_df['cluster'] == 1]
df2 = cluster_df[cluster_df['cluster'] == 2]
df3 = cluster_df[cluster_df['cluster'] == 3]

In [ ]:
'''
spectrogram image size=(224, 224)

ideal params (leveraged with resnet18 model)
  batch_size:
    value: 64
  sample_rate:
    value: 44100
  hop_length:
    value: 308 
  win_length:
    value: 2205
  n_mels:
    value: 224
  n_fft:
    value: 4096
  normalize:
    value: True
  mix_up:
    value: 0.1
  f_max:
    value: 18000
  arch:
    value: resnet18
  n_epochs:
    values: [10, 20, 80] 
  trial_num:
    values: [1, 2, 3, 4, 5]
  fold:
    values: [1, 2, 3, 4, 5]

for audio classification with resnet18 pre-trained
about 89.54% accuracy, in 80 epochs, little over 14 minutes, although I think this is with some heavy rental fees

'''

In [ ]:
# # tensorflow syntax here
# def self_attention(in_shape, ch, k=8):
#   height, width, channel = in_shape
#   x = layers.Input(shape = [height, width, channel])
	
#   f = layers.Conv2D(ch // k, kernel_size = (1, 1), strides = (1, 1), padding='same', use_bias=True)(x)
#   f = layers.MaxPooling2D()(f)
#   f = layers.Reshape((-1, f.shape[-1]))(f)
	
#   g = layers.Conv2D(ch // k, kernel_size = (1, 1), strides = (1, 1), padding='same', use_bias=True)(x)
#   g = layers.Reshape((-1, g.shape[-1]))(g)
	
#   h = layers.Conv2D(ch // 2, kernel_size = (1, 1), strides = (1, 1), padding='same', use_bias=True)(x)
#   h = layers.MaxPooling2D()(h)
#   h = layers.Reshape((-1, h.shape[-1]))(h)
	
#   s = tf.matmul(g, f, transpose_b=True)
#   s = keras.layers.Softmax()(s)
		
#   o = tf.matmul(s, h)

#   o = layers.Reshape((height, width, ch // 2))(o)
#   o = layers.Conv2D(channel, kernel_size = (1, 1), strides = (1, 1), padding='same', use_bias=True)(o)
#   o = Scalar()(o)
#   o = o + x

#   SA = keras.Model(inputs=x, outputs=o)
	
#   return SA


# class Scalar(layers.Layer):
#   def __init__(self):
#     super(Scalar, self).__init__()

#   def build(self, input_shape):
#     self.gamma = tf.Variable(initial_value=tf.zeros(1), trainable=True)
#     self._trainable_weights=[self.gamma]

#   def call(self, inputs):
#     return layers.Rescaling(self.gamma)(inputs)

In [ ]:
def conv_model():
  input = layers.Input(shape=(256, 128))

  image = SpectralNorm(layers.Conv2D(128, (5, 5), strides=(1, 1), padding='same', input_shape=[256, 128, 1]))(input)
  image = layers.LeakyReLU(image)
  image = layers.Dropout(0.3)(image)

